# Visualizing
Inputs: Query, Structured Json

### Setup

In [40]:
!pip install anthropic
!pip install plotly

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 17.9 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [1]:
import pandas as pd
import json
import re
import anthropic
import os
import matplotlib.pyplot as plt
import plotly.express as px

import openai

Define API key for Anthropic:

In [2]:
# fill your key here:
# os.environ['ANTHROPIC_API_KEY'] = "insert_your_anthropic_api_key_here"

# and disable this line here:
os.environ['ANTHROPIC_API_KEY'] = os.getenv("ANTHROPIC_API_KEY")

client = anthropic.Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

### Define Query + Dataset

In [3]:
path_structured = 'GeneratedAnswers/Q2_structured.json'
user_query = 'Create a stacked bar chart that shows the count of plans including/not-including detailed instructions, grouped by year'

Extract Metadata date of plan:

In [8]:
# Step 1: Load the JSON
with open('../metadata/plans_metadata.json', 'r', encoding='utf-8') as f:
    metadata = json.load(f)

# Step 2: Extract year as a dict of plan_id → year (last 4 digits from "year" field)
year_data = {}
for plan_id, info in metadata.items():
    year_str = info.get("year", "")
    match = re.search(r'20\d{2}', year_str)  # matches a 4-digit year starting with 20
    if match:
        year_data[plan_id] = int(match.group(0))
    else:
        print(f"No valid year found for plan_id: {plan_id}, raw value: {year_str}")

# Step 3: Convert to DataFrame
df_years = pd.DataFrame(list(year_data.items()), columns=["plan_id", "year"])


Load JSON:

In [9]:
# Load the structured JSON file
with open(path_structured, 'r', encoding='utf-8') as f:
    data = json.load(f)

# Convert to DataFrame
df = pd.DataFrame(data)

# Display the first few rows
df.head()

,plan_id,contains_detailed_plan_instructions
0,507-0668632,True
1,507-0680413,True
2,507-0621870,False
3,507-0609446,True
4,507-0444091,True


Merge with the metadata:

In [10]:
df["plan_id"] = df["plan_id"].astype(str)  # ensure string type for join
df = df.merge(df_years, on="plan_id", how="left")

# Show the updated df
df.head()

,plan_id,contains_detailed_plan_instructions,year
0,507-0668632,True,2020
1,507-0680413,True,2020
2,507-0621870,False,2023
3,507-0609446,True,2022
4,507-0444091,True,2017


## Step 1: Prepare Question 

Convert the query from free text to a format fitting an LLM

In [11]:
def prepare_prompt(df, user_query, n_rows_preview=5):
    """
    Prepare a prompt for the GPT API to generate a visualization code.
    Includes a preview of the dataset and a user query.
    """
    preview = df.head(n_rows_preview).to_csv(index=False)
    prompt = f"""You are a Python data analysis and visualization specialist.

    Here is a preview of the dataset:
    {preview}

    Task: {user_query}

    Please write Python code using plotly.express library, or matplotlib (or seaborn if appropriate) to answer the task with a 
    corresponding beautiful interactive visualization - answering the described query. 
    Assume the full dataset is already loaded into a pandas DataFrame variable named `df` outside of the scope of your task at hand. 
    Do not define/load 'df' in your code. Just focus on the visualization part.
    Make sure the code runs without error and shows the plot.

    Check the output visualization for outliers and if there are any, use the IQR method to remove them.
    For Histograms, use as many bins as possible.

    Use a black background and bright pastel colors for the plot (such as #A3BFD9/#BFD8B8/#C4C3D0/#F6B7A9), and white text.

    First, organize the data from 'df' in a way that will enable answering the query described in the task,
    make use of the preview of the dataset above. Then, create a **clean and interactive** visualization that allows zooming, 
    hovering, and panning.
    """
        
    return prompt


In [12]:
prompt = prepare_prompt(df, user_query, n_rows_preview=5)
print(prompt)

You are a Python data analysis and visualization specialist.

    Here is a preview of the dataset:
    plan_id,contains_detailed_plan_instructions,year
507-0668632,True,2020
507-0680413,True,2020
507-0621870,False,2023
507-0609446,True,2022
507-0444091,True,2017


    Task: Create a stacked bar chart that shows the count of plans including/not-including detailed instructions, grouped by year

    Please write Python code using plotly.express library, or matplotlib (or seaborn if appropriate) to answer the task with a 
    corresponding beautiful interactive visualization - answering the described query. 
    Assume the full dataset is already loaded into a pandas DataFrame variable named `df` outside of the scope of your task at hand. 
    Do not define/load 'df' in your code. Just focus on the visualization part.
    Make sure the code runs without error and shows the plot.

    Check the output visualization for outliers and if there are any, use the IQR method to remove them.
    F

## Step 2: Send to Claude

In [369]:
def call_claude(prompt, model="claude-3-haiku-20240307"):
    response = client.messages.create(
        model=model,
        max_tokens=1024,
        temperature=0.3,
        messages=[
            {"role": "user", "content": prompt}
        ]
    )
    return response.content[0].text  # Returns the first text block

In [411]:
responseRaw = call_claude(prompt)

In [372]:
print(responseRaw)

Sure, here's the Python code using Plotly Express to create a boxplot of the area values across the entire set of plans over the course of the years, with the requested customizations:

```python
import pandas as pd
import plotly.express as px

# Assuming the dataset is already loaded into a pandas DataFrame called 'df'
df = pd.DataFrame({
    'plan_id': ['507-0444091', '507-0595439', '507-0322842', '507-0892091', '507-0609180'],
    'area_dunams': [3.777, 27.221, 30.357, 19.705, 6.278],
    'year': [2017, 2024, 2021, 2024, 2019]
})

# Create the boxplot
fig = px.box(df, x='year', y='area_dunams', color='year', 
             title='Area Values Across Plans Over the Years',
             template='plotly_dark', color_discrete_sequence=['#A3BFD9', '#BFD8B8', '#C4C3D0', '#F6B7A9'])

# Customize the plot
fig.update_layout(
    plot_bgcolor='black',
    paper_bgcolor='black',
    font_color='white',
    xaxis_title='Year',
    yaxis_title='Area (dunams)',
    xaxis_tickangle=-45,
    margin=

## Step 3: Run Code

Parse it first:

In [366]:
def extract_code_block(text):
    """
    Extract the first Python code block (```python ... ```) from a string.
    """
    match = re.search(r"```(?:python)?\n(.*?)```", text, re.DOTALL)
    if match:
        return match.group(1)
    else:
        print("⚠️ No code block found.")
        return None


In [412]:
code = extract_code_block(responseRaw)

if code:
    # Define the environment the code should run in (e.g., with access to `df`)
    local_env = {"df": df, "plt": plt, "pd": pd}

    try:
        exec(code, {}, local_env)
    except Exception as e:
        print(f"❌ Error while executing code:\n{e}")
